**By: Jacob van Steyn, ...**

# DS3010 Project : Twitter political analysis


In [ ]:
%%capture
!pip install tweepy pymongo alive-progress pandas matplotlib python-dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# VARIABLES
SHOULD_CLEAR_DB = False
SHOULD_SCRAPE = False


In [ ]:
#----------------------------------------------
# Using dotenv (loads from a file called '.env') to keep secrets safe
load_dotenv("/content/drive/MyDrive/DS 3010 - Group Project/.env")
MONGO_USERNAME = os.getenv("MONGO_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
MONGO_URL = os.getenv("MONGO_URL")

client = pymongo.MongoClient(f"mongodb+srv://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_URL}/?retryWrites=true&w=majority")

dblist = client.list_database_names()
if "DS3010" in dblist:
    print("The database already exists!")
else:
    print("Database was created.")

#dblist = client.list_database_names()
#if "DS3010" in dblist:
#    print("The database already exists!")
#else:
#   print("Database was created.")

# As mongodb won't create the database until a document is created, let's insert & delete a fake one!
db = client.DS3010
celeb_tweets = db.celeb_tweets

if SHOULD_CLEAR_DB:
  document = celeb_tweets.insert_one({"test":"temp document!"})
  print(f"Num Tweets Stored: {celeb_tweets.count_documents({})}")


  # Clear DB
  celeb_tweets.delete_many({})
  print(f"Num Tweets Stored: {celeb_tweets.count_documents({})}")





In [ ]:
#----------------------------------------------
import json
import time
import re

import tweepy
from alive_progress import alive_bar
import pandas as pd
import matplotlib.pyplot as plt


# Initialize our tweepy client
def setup():
    # Twitter API key and secret
    
    consumer_key = os.getenv("TWITTER_CONSUMER_KEY")
    consumer_secret = os.getenv("TWITTER_CONSUMER_SECRET")
    
    # Handling authentication with Twitter
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    # Create a wrapper for the Twitter API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api
    
# Helper function for handling handling rate limits and getting each page of results
    # Use alive_bar to display a progress bar as tweets are fetched
def limit_handler(cursor, total):
    with alive_bar(total, force_tty=True) as bar:
        while True:
            try:
                yield cursor.next()
                bar()
            except StopIteration:
                if (bar.current != total):
                    print(f"[Stopped!] Found {bar.current} tweets for specifed term!")
                break
                
api = setup()

TweepError: ignored

In [ ]:
# Read accounts from CSV
import csv

user_map = {}
with open("/content/drive/MyDrive/DS 3010 - Group Project/accounts.csv", "r") as file:
  # reading the CSV file
  csvFile = csv.reader(file)
 
  # displaying the contents of the CSV file
  for lines in csvFile:
        if lines[0] and lines[1]:
          user_map[lines[0][1:]] = lines[1] == "D"

print(len(user_map))

390


In [ ]:
from tweepy.parsers import TweepError
# Search term (omitting retweets)
count = 270

print("Setup! Retrieving Tweets now...")

# Search for tweets using Tweepy 
# Have first 45

deleted_users = []

if SHOULD_SCRAPE:
  for i, (user, val) in enumerate(user_map.items()):
    print(f"User: {user}, Index: {i}, isDem? {val}")
    search = limit_handler(tweepy.Cursor(api.user_timeline,
                            screen_name=user,
                            exclude_replies=True,
                            include_rts=False,
                            count=90).items(count), count)
    
    returned_tweets = []
    try: 
      for result in search:
          returned_tweets.append(result._json)

      if val:
        print(f"Inserting Tweets to MongoDB...\nCurrent # Dem Tweets Stored: {celeb_tweets.count_documents({})}")
        celeb_tweets.insert_many(returned_tweets)
        print(f"New Celeb Tweets Stored: {celeb_tweets.count_documents({})}")
    except TweepError:
      deleted_users.append(user)
      print("USER NOT FOUND!")
      continue
else:
  print("Not Scraping!")
  
  

# Process the results from the search using Tweepy




Setup! Retrieving Tweets now...
User: BarackObama, Index: 0, isDem? True
|████████████████████████████████████████| 270/270 [100%] in 2.4s (101.54/s)                                            
Inserting Tweets to MongoDB...
Current # Dem Tweets Stored: 0
New Dem Tweets Stored: 270
User: realDonaldTrump, Index: 1, isDem? False
|████████████████████████████████████████| 270/270 [100%] in 1.5s (165.20/s)                                            
Inserting Tweets to MongoDB...
Current # Rep Tweets Stored: 0
New Dem Tweets Stored: 270
User: SpeakerPelosi, Index: 2, isDem? True
|████████████████████████████████████████| 270/270 [100%] in 1.3s (200.00/s)                                            
Inserting Tweets to MongoDB...
Current # Dem Tweets Stored: 270
New Dem Tweets Stored: 540
User: RepAndyBiggsAZ, Index: 3, isDem? False
|████████████████████████████████████████| 270/270 [100%] in 2.0s (138.94/s)                                            
Inserting Tweets to MongoDB...
Current #

In [ ]:
#----------------------------------------------

# Deleted accounts: ['RepKClark', 'WhipClyburn', 'RepTomEmmer', 'o official', 'RepKiggans', 'RepLuttrell_TX8', 'teammoulton', 'RepMullin']
missing_accounts = db.missing_accounts
print(deleted_users)
#missing_accounts.insert_many({usr: user_map[usr] for usr in missing_accounts})

# Get total number of tweets

# Save tweet text into republican and democratic lists


celeb_tweets = []



['RepKClark', 'WhipClyburn', 'RepTomEmmer', 'o official', 'RepKiggans', 'RepLuttrell_TX8', 'teammoulton', 'RepMullin']


TypeError: ignored